In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report,accuracy_score
import pandas as pd
from sklearn.datasets import load_iris

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [3]:
import os
from keras.preprocessing import image
import cv2

global base_dir, image_paths, target_size
base_dir = './dataset'
images_dir = os.path.join(base_dir,'image')
label_filename = 'label.csv'

image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
target_size = None
# Opening the label file
label_file = open(os.path.join(base_dir, label_filename), 'r')
# Reading the lines
label_lines = label_file.readlines()
#print(image_paths)
tumor_labels = {line.split(',')[0] : line.split(',')[1] for line in label_lines[1:]}
for k, v in tumor_labels.items():
    tumor_labels[k] = v.replace('\n', '')
#tumor_labels[:,1] = {line.replace('\n', '') for line in tumor_labels[1:]}
for k in tumor_labels:
    if 'no_tumor' in tumor_labels[k]:
        tumor_labels[k] = [0, 0, 0, 1]
    elif 'meningioma_tumor' in tumor_labels[k]:
        tumor_labels[k] = [1, 0, 0, 0]
    elif 'glioma_tumor' in tumor_labels[k]:
        tumor_labels[k] = [0, 1, 0, 0]
    elif 'pituitary_tumor' in tumor_labels[k]:
        tumor_labels[k] = [0, 0, 1, 0]
    else:
        tumor_labels[k] = [0, 0, 0, 0]
    #tumor_labels[k].contains() == 
#print(tumor_labels)

if os.path.isdir(images_dir):
    all_features = []
    #         all_features = []
    all_labels = []
    for img_path in image_paths:
         #No need for this 
            # file_name= img_path.split('/')[-1]
        #file_name= img_path.split('.')[1].split('/')[-1] + '.jpg'
        file_name= img_path.split('\\')[-1]
        # load image
        img = image.img_to_array(image.load_img(img_path))
        img = cv2.resize(img, (456, 456))
        #img = image.img_to_array(image.load_img(img_path, color_mode='grayscale', target_size=target_size, interpolation='bicubic'))
        img = np.reshape(img, (456,456, 3))
        # Getting rid of rgb data to greyscale
        #img = np.squeeze(img)
        if img is not None:
            all_features.append(img)
            all_labels.append(tumor_labels[file_name])


In [4]:
x = np.array(all_features)
y = np.array(all_labels)
# print(xtrain.shape)
#x = np.squeeze(x)
#print(xtrain.shape)
#x_g /= 255
print(x.shape)

(3000, 456, 456, 3)


In [5]:
x /= 255
#x = np.reshape(x, (x.shape[0], (512*512)))
xtrain, xtest, ytrain, ytest = train_test_split(x, y, train_size=0.80, random_state=10) 

In [6]:
print(x.shape)
print(y.shape)
print(y)

(3000, 456, 456, 3)
(3000, 4)
[[1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 ...
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]]


In [10]:
print(x_g.shape)  # (64, 224, 224)
rgb_batch = np.repeat(x_g[512, 512, np.newaxis], 3, -1)
print(rgb_batch.shape)  # (64, 224, 224, 3)

AttributeError: 'list' object has no attribute 'shape'

In [7]:
#from tensorflow.keras.applications import EfficientNetB5

from keras_efficientnets import EfficientNetB5

model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(456,456,3))


In [8]:
model1 = model.output
model1 = keras.layers.GlobalAveragePooling2D()(model1)
model1 = keras.layers.Dropout(rate=0.5)(model1)
model1 = keras.layers.Dense(32, activation='relu')(model1)
model1 = keras.layers.Dropout(rate=0.5)(model1)
model1 = keras.layers.Dense(4, activation='softmax')(model1)
model1 = keras.models.Model(inputs=model.input, outputs=model1)


In [9]:
model1.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 456, 456, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 228, 228, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 228, 228, 48) 192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
swish_1 (Swish)                 (None, 228, 228, 48) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [10]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
final = model1.fit(xtrain, ytrain, batch_size=2, epochs=5, validation_data=(xtest, ytest))

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 602s 251ms/step - loss: 1.3897 - accuracy: 0.3058 - val_loss: 4.8721 - val_accuracy: 0.3050
Epoch 2/5
2400/2400 [==============================] - 559s 233ms/step - loss: 1.3578 - accuracy: 0.2933 - val_loss: 1.3031 - val_accuracy: 0.3633
Epoch 3/5
2400/2400 [==============================] - 559s 233ms/step - loss: 1.3496 - accuracy: 0.2962 - val_loss: 1.3190 - val_accuracy: 0.3517
Epoch 4/5
2400/2400 [==============================] - 555s 231ms/step - loss: 1.3596 - accuracy: 0.2875 - val_loss: 1.3529 - val_accuracy: 0.2917
Epoch 5/5
2400/2400 [==============================] - 554s 231ms/step - loss: 1.3588 - accuracy: 0.2821 - val_loss: 1.3549 - val_accuracy: 0.2867
